In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Naive Bayes for Text Classification

## Introduction

The dataset contains SMS messages labeled as *ham* (non-spam) or *spam*. We'll use scikit-learn's Naive Bayes Classifier to predict these labels.

Originally published here: [link](https://github.com/Bhasfe/ml-algorithms/tree/master/Naive%20Bayes)

### How does it work ?

Naive Bayes is an algorithm which is commonly used in natural language processing (NLP) tasks such as spam filtering, sentiment analysis, classification, recommendation. It is based on *Bayes' Theorem* as shown below. 

<img src="https://github.com/Bhasfe/ml-algorithms/blob/master/Naive%20Bayes/bayesian.png?raw=true" width="300px" height="300px" align="left" /><br><br><br><br><br>

*P(A|B)* = probability of A if B occurs <br>
*P(B|A)* = probability of B if A occurs <br>
*P(A)* = probability of A <br>
*P(B)* = probability of A <br>

A Naive Bayes classifier gives us the conditional probabilities of events occur related to each other by using Bayes' Theorem. If you would like to deep into, please visit [here](https://www.kaggle.com/prashant111/naive-bayes-classifier-tutorial).<br>
<br>
In scikit-learn, there are 3 types naive bayes algorithms.

**Gaussian Naive Bayes:** It works with continuous attributes, it assumes the data normally distributed (Gaussian Distribution) <br><br>
**Multinomial Naive Bayes:** It works with frequencies of features.It is used text classification or <br><br>
**Bernoulli Naive Bayes:** It works with multinomial binary variables, usually used for text classification like multinomial<br><br>

We'll use **Multinomal** in this notebook.

### Basics of Natural Language Processing

Since we'll use *Naive Bayes* in a *text classification* task, I would like to explain briefly some concepts that are we going to use.<br>

**Lemmatization:** It is a process of make the same words in their stem. For example run, ran, running are different in terms of Python. We lemmatiza the word to get run.<br><br>
**Stop words:** The words that are not important in terms of the context<br><br>
**Tokenization:** The process of extracting words in a sentence by spaces and punctuations. In this project we'll use nltk's [word_tokenize](https://www.nltk.org/api/nltk.tokenize.html)<br><br>
**Bag Of Words:** BoW is the representation of text data in a numerical way that machine learning algoritms can work with. <br><br>
**Tf-idf (Term Frequency - Inverse Term Frequency):** It is a statistical concept to be used to get importance of words in corpus. We'll use scikit-learn's *TfidfVectorizer*. The vectorizer will calculate the weight of each word in corpus and will return a tf-idf matrix. You can find further information [here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)<br><br>
<img src="https://github.com/Bhasfe/ml-algorithms/blob/master/Naive%20Bayes/tfidf.png?raw=true" width="300px" height="300px" align="left" /><br><br><br><br><br>
*td* = Term frequency (number of occurance each i in j)<br>
*df* = Document frequency <br>
*N* = Number of documents<br>
*w* = tf-idf weight
for each *i* and *j* (document).

### Requirements

Before start the project. We have to install the necessary libraries via following commands.

`pip install -U scikit-learn`

`pip install wordcloud`

`pip install --user -U nltk`

In [ ]:
# Fundamentals
import matplotlib.pyplot as plt
import seaborn as sns

# Import NLTK to use its functionalities on texts
"""DO NOT forget to download followings if you do not have
# nltk.download('punkt')
#nltk.download('wordnet')
"""
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# We will visualize the messages with a word cloud
from wordcloud import WordCloud

# Multinomial Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB

# Import Tf-idf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Import the Label Encoder
from sklearn.preprocessing import LabelEncoder

# Import the train test split
from sklearn.model_selection import train_test_split

# To evaluate our model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

# I will keep the resulting plots
%matplotlib inline

# Enable Jupyter Notebook's intellisense
%config IPCompleter.greedy=True

## 1. Exploratory Data Analysis

Let's start with importing the data into a pandas DataFrame called data

In [ ]:
# Load the data
data = pd.read_csv('/kaggle/input/spam-text-message-classification/SPAM text message 20170820 - Data.csv')

Now we can look at main features of the dataset

In [ ]:
# Display firt five rows
display(data.head())

# Display the summary statistics
display(data.describe())

# Print the info
print(data.info())

Our dataset has **Message** and **Category** columns which consist of *object* data type. There are 5572 messages. We have to check whether the category data is *balanced or not*.

In [ ]:
# Print the counts of each category
print(data['Category'].value_counts())

print()

# Print the proportions of each category
print(data['Category'].value_counts(normalize=True))

# Visualize the Categories
sns.countplot(data['Category'])
plt.title("Category Counts")
plt.show()

As we can see above, the dataset is unbalanced. We have to consider this when we build our model. As well as, we need to encode the labels to use our machine learning model. To achieve this we'll use the Label Encoder from scikit-learn

In [ ]:
# Initialize the Label Encoder.
le = LabelEncoder()

# Encode the categories
data['Category_enc'] = le.fit_transform(data['Category'])

# Display the first five rows again to see the result
display(data.head())

# Print the datatypes
print(data.dtypes)

We got the encoded categories. Now, *0 = ham* and *1 = spam*.

## 2. Feature Engineering

Before build the classification model, let's explore the data. First we'll compare the word counts of messages in each category

In [ ]:
# Store the number of words in each messages
data['word_count'] = data['Message'].str.split().str.len()

# Print the average number of words in each category
print(data.groupby('Category')['word_count'].mean())

# Visualize the distribution of word counts in each category
sns.distplot(data[data['Category']=='spam']['word_count'], label='Spam')
sns.distplot(data[data['Category']=='ham']['word_count'], label='Ham'),
plt.legend()
plt.show()

Apparently, spams tend to contain more number of words. What about the most common words in each ?

In [ ]:
# Make the letters lower case and tokenize the words
tokenized_messages = data['Message'].str.lower().apply(word_tokenize)

# Print the tokens to see how it looks like
print(tokenized_messages)

There some non-alphanumeric characters ( * ' " - ) and stop words like a, and, the etc. Let's discard them

In [ ]:
# Define a function to returns only alphanumeric tokens
def alpha(tokens):
    """This function removes all non-alphanumeric characters"""
    alpha = []
    for token in tokens:
        if str.isalpha(token) or token in ['n\'t','won\'t']:
            if token=='n\'t':
                alpha.append('not')
                continue
            elif token == 'won\'t':
                alpha.append('wont')
                continue
            alpha.append(token)
    return alpha

# Apply our function to tokens
tokenized_messages = tokenized_messages.apply(alpha)

print(tokenized_messages)

It's time to deal with the stop words. We've already imported stopwords from nltk.

In [ ]:
# Define a function to remove stop words
def remove_stop_words(tokens):
    """This function removes all stop words in terms of nltk stopwords"""
    no_stop = []
    for token in tokens:
        if token not in stopwords.words('english'):
            no_stop.append(token)
    return no_stop

# Apply our function to tokens
tokenized_messages = tokenized_messages.apply(remove_stop_words)

print(tokenized_messages)

Moreover, we need to lemmatize the words. We've already imported WordNetLemmatizer from nltk.

In [ ]:
# Define a function to lemmatization
def lemmatize(tokens):
    """This function lemmatize the messages"""
    # Initialize the WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    # Create the lemmatized list
    lemmatized = []
    for token in tokens:
            # Lemmatize and append
            lemmatized.append(lemmatizer.lemmatize(token))
    return " ".join(lemmatized)

# Apply our function to tokens
tokenized_messages = tokenized_messages.apply(lemmatize)

print(tokenized_messages)

Let's replace Message column with tokenized_messages

In [ ]:
# Replace the columns with tokenized messages
data['Message'] = tokenized_messages

# Display the first five rows
display(data.head())

Visualize the words mostly used in each type of messages.

In [ ]:
# Get the spam messages
spam = data[data['Category']=='spam']['Message'].str.cat(sep=', ')

# Get the ham messages
ham = data[data['Category']=='ham']['Message'].str.cat(sep=', ')

# Initialize the word cloud
wc = WordCloud(width = 500, height = 500, min_font_size = 10, background_color ='white')

# Generate the world clouds for each type of message
spam_wc = wc.generate(spam)

# plot the world cloud for spam                     
plt.figure(figsize = (5, 5), facecolor = None) 
plt.imshow(spam_wc) 
plt.axis("off") 
plt.title("Common words in spam messages")
plt.tight_layout(pad = 0) 
plt.show() 
ham_wc = wc.generate(ham)

# plot the world cloud for spam                       
plt.figure(figsize = (5, 5), facecolor = None) 
plt.imshow(ham_wc) 
plt.axis("off")
plt.title("Common words in ham messages")
plt.tight_layout(pad = 0) 
plt.show() 

It seems that ham messages contain lot's of abbreviation and informal words. Spams tend to contain mostly *free, mobile, cole, text*

## 3. Build the model

First select the our features and the target.

In [ ]:
# Select the features and the target
X = data['Message']
y = data['Category_enc']

We need to split our data into train and test sets. We'll use **stratify** parameter of train_test_split since our data is unbalanced

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=34, stratify=y)

Now, we can get the tf-idf by using scikit-learn's TfidfVectorizer. Tf-idf stands for *term frequency - inverse document frequency*. It is commonly used in Natural Language Processing to determine the most important words in the document.

In [ ]:
# Create the tf-idf vectorizer
vectorizer = TfidfVectorizer(strip_accents='ascii')

# First fit the vectorizer with our training set
tfidf_train = vectorizer.fit_transform(X_train)

# Now we can fit our test data with the same vectorizer
tfidf_test = vectorizer.transform(X_test)

Finally, we can build the our machine learning model.

In [ ]:
# Initialize the Multinomial Naive Bayes classifier
nb = MultinomialNB()

# Fit the model
nb.fit(tfidf_train, y_train)

# Print the accuracy score
print("Accuracy:",nb.score(tfidf_test, y_test))

It gives us approximately 96% accuracy. 

## 4. Evaluate the model

The accuracy score is not enought to say our model's performance great. We need to do more calculation to be make sure.

In [ ]:
# Predict the labels
y_pred = nb.predict(tfidf_test)

# Print the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix\n")
print(cm)

# Print the Classification Report
cr = classification_report(y_test, y_pred)
print("\n\nClassification Report\n")
print(cr)


# Print the Receiver operating characteristic Auc score
auc_score = roc_auc_score(y_test, y_pred)
print("\nROC AUC Score:",auc_score)

# Get probabilities.
y_pred_proba = nb.predict(tfidf_test)

# Get False Positive rate, True Positive rate and the threshold
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

# Visualize the ROC curve.
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('FP Rate')
plt.ylabel('TP Rate')
plt.title('ROC')
plt.show()

Area under the ROC curve gave us lower but it still looks fine. According to confusion matrix and classification report our model is perfect to detect spams but not hams

This is the and of the notebook. I hope it can be helpful to understand using *Naive Bayes* classifier in Python.